# Data Cleaning
The cleaning process is organized into sections, each corresponding to a different dataset (`.csv` file). Each section includes the following steps:

1. **Data Understanding**: Initial exploration of the dataset.
2. **Data Cleaning**: Handling of missing values (NaN), removal of duplicates, setting correct data types, and renaming columns. <br>
   *(Optional)* **Deep Clean**: Custom cleaning steps applied to a specific dataset, if necessary.
3. **Final Result**: Displays the cleaned dataset and saves it to a new `.csv` file.

All the datasets combined have a total size of ~1Gb and can be all uploaded in memory at the same time on almost every PC.

First, import the necessary libraries and set up any required options.

In [ ]:
import pandas as pd
import numpy as np

from utils.utils import find_matching

# Set to True to print cleaned data into new csv
PRINT_CSV = False

## Movies

In [ ]:
# Import 'movies.csv' dataset
movies_df = pd.read_csv('datasets/movies.csv')

### 1. Data Understanding

In [ ]:
movies_df.head()

In [ ]:
movies_df.shape

In [ ]:
movies_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
movies_df = movies_df.rename(columns={'name': 'title', 'minute': 'duration_in_minutes', 'date': 'release_year'})
print(f"Movies dataset columns: {', '.join(movies_df.columns)}")

In [ ]:
# Check for null values
movies_df.isna().sum()

There are null values in most of the columns.
The fields '**release_year**', '**tagline**', '**description**', '**duration_in_minutes**' and '**rating**' don't cause any problems so we'll keep them, but the few movies that are without a title can't be used and will be removed.

In [ ]:
# Removing rows with null title
no_title = movies_df[movies_df['title'].isna()]
movies_df = movies_df.dropna(subset=['title'])

print("Movies dataset without title:")
no_title.head()

In [ ]:
# Check for duplicate rows
print(f"There are {movies_df.duplicated().sum()} duplicated rows")

In [ ]:
# Setting the correct type for columns
movies_df['release_year'] = movies_df['release_year'].astype('Int64')
movies_df['duration_in_minutes'] = movies_df['duration_in_minutes'].astype('Int64')
movies_df[['release_year', 'duration_in_minutes']].dtypes

In [ ]:
# Check if 'id' column has unique values
print(f"'id' duplicates: {movies_df[movies_df['id'].duplicated()].shape[0]}")
movies_df = movies_df.set_index("id")

The '**id**' field is the unique identifier of a movie, so it's been set as the index.

#### Deep Clean
Let's look inside some columns to see most frequent values:

In [ ]:
movies_df['description'].value_counts().head(10)

Many descriptions seem to have a description like "Plot Unavailable" or similar instead of a null value. The other fields seem fine.<br>
Let's try to fix as many as possible (fixing only the most frequent variation, not 100% accurate).

In [ ]:
from utils.utils import null_movie_description_keywords

# Find null description variation
result = find_matching(movies_df, 'description', null_movie_description_keywords, max_length=30)
matches = result.copy()

# Fill with NaN values the result obtained
result['description'] = np.nan

# Manual check to be sure to not overwrite real descriptions
matches['description'].value_counts()

### 3. Final Result
All datasets reference the **movies** dataset. A movie is uniquely identified by his **id** and a movie id has multiple occurrences in other datasets. A movie has a title, a tagline, a description, the release year, the duration and a rating. Only the title is mandatory and all the other attributes could be missing.

In [ ]:
movies_df.head()

In [ ]:
movies_df.shape

## Languages

In [ ]:
# Import 'languages.csv' dataset
lang_df = pd.read_csv('datasets/languages.csv')

### 1. Data Understanding

In [ ]:
lang_df.head()

In [ ]:
lang_df.shape

In [ ]:
lang_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
lang_df = lang_df.rename(columns={'id': 'movie_id'})
print(f"Languages dataset columns: {', '.join(lang_df.columns)}")

In [ ]:
# Check for null values
lang_df.isna().sum()

In [ ]:
# Check for duplicate rows
print(f"There are {lang_df.duplicated().sum()} duplicated rows")

In [ ]:
# Setting the category data type for column 'type'
print(f"types: {', '.join(lang_df['type'].unique())}")
lang_df['type'] = lang_df['type'].astype('category')

The `type` field has only 3 possible values, so we can set it as a categorical type.

### 3. Final Result
The languages dataset is directly connected to the movies dataset with the 'movie_id' column. There are more languages rows than movies rows, because a movie can have multiple languages connected. Also, not all movie must have a language defined. A language can be of three types:
- *Language*: Could be a generic language associated with the movie, used when there is a single dominant language.
- *Primary Language*: Could be the main or original language of the movie.
- *Spoken Language*: Could be all the languages actually used in the movie's dialogues.

In [ ]:
lang_df.head()

In [ ]:
lang_df.shape

## Actors

In [ ]:
# Import 'actors.csv' dataset
actors_df = pd.read_csv('datasets/actors.csv')

### 1. Data Understanding

In [ ]:
actors_df.head()

In [ ]:
actors_df.shape

In [ ]:
actors_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
actors_df = actors_df.rename(columns={'id': 'movie_id'})
print(f"Actors dataset columns: {', '.join(actors_df.columns)}")

In [ ]:
# Check for null values
actors_df.isna().sum()


Despite the high number of null values in the `role` field, the rows will be maintained because they still contain information about the actor's `name`. An actor without a name is unusable information, so the corresponding rows will be removed.

In [ ]:
# Removing actors without name
no_name = actors_df[actors_df['name'].isna()]
actors_df = actors_df.dropna(subset=['name'])
no_name

In [ ]:
# Check for duplicate rows
print(f"There are {actors_df.duplicated().sum()} duplicated rows")
actors_duplicates = actors_df[actors_df.duplicated(keep=False)].head(6)

# Dropping the duplicates
actors_df = actors_df.drop_duplicates()

actors_duplicates

Completely duplicated rows are clearly an error and can be removed.

#### Deep Clean

In [ ]:
actors_df['role'].value_counts().head(10)

The role column has many "Self" role variations let's look more deeply.

In [ ]:
from utils.utils import self_actor_role_keywords

# Find self variation
result = find_matching(actors_df, 'role', self_actor_role_keywords)
print(f"Rows contains 'self' variations: {result['role'].shape[0]}")
result['role'].value_counts().head()

There are over 300.000 values that are similar to "Self", but many of them contains also other information as "Self - Presenter" or "Self - Guest". Overwriting all those values could result in a loss of information, so they won't be overwritten in the cleaned dataset, but they might be when visualizing the data for statistical purposes.

In [ ]:
# Reset indexing after removing rows
actors_df = actors_df.reset_index(drop=True)

### 3. Final Result
The actors dataset is directly connected to the movies dataset and has almost six times the number of rows as the movies dataset. Also, a movie can have no actors connected. The same actor can have multiple occurrences in the dataset because he can appear in more than on movie. <br> An actor is identified solely by his full name, stored in a single field.


In [ ]:
actors_df.head()

In [ ]:
actors_df.shape

## Countries

In [ ]:
# Import 'countries.csv' dataset
countries_df = pd.read_csv('datasets/countries.csv')

### 1. Data Understanding

In [ ]:
countries_df.head()

In [ ]:
countries_df.shape

In [ ]:
countries_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
countries_df = countries_df.rename(columns={'id': 'movie_id'})
print(f"Countries dataset columns: {', '.join(countries_df.columns)}")

In [ ]:
# Check for null values
countries_df.isna().sum()

In [ ]:
# Check for duplicate rows
print(f"There are {countries_df.duplicated().sum()} duplicated rows")

### 3. Final Results
The **countries** dataset is directly connected to the movies dataset with the 'movie_id' column. This dataset contains all the countries where the movies were produced.


In [ ]:
countries_df.head()

In [ ]:
countries_df.shape

## Crew

In [ ]:
# Import 'crew.csv' dataset
crew_df = pd.read_csv('datasets/crew.csv')

### 1. Data Understanding

In [ ]:
crew_df.head()

In [ ]:
crew_df.shape

In [ ]:
crew_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
crew_df = crew_df.rename(columns={'id': 'movie_id', 'name': 'crew_member_name'})
print(f"Crew dataset columns: {', '.join(crew_df.columns)}")

In [ ]:
# Check for null values
crew_df.isna().sum()

The existing `NaN` value was not removed because it is minimal compared to the overall size of the dataset, and removing it would not significantly impact the analysis. Similarly, values such as *Unknown* or *Anonymous* were kept because they account for less than 0.1% of the data and do not affect the overall results.

In [ ]:
# Check for duplicate rows
print('Duplicated rows:', crew_df.duplicated().sum())
crew_duplicates = crew_df[crew_df.duplicated(keep=False)].head()

# Dropping the duplicates
crew_df = crew_df.drop_duplicates()

crew_duplicates

Completely duplicated rows are clearly an error and can be removed.

#### Deep Clean

In [ ]:
# TODO: deep clean to remove escape from name column

In [ ]:
# Reset indexing after removing rows
crew_df = crew_df.reset_index(drop=True)

### 3. Final Results
The **crew** dataset is connected to the movies dataset through the `movie_id` column. It includes the names of all crew members along with their roles. <br>
A crew member can have different roles in the same movies and can appear in more than one movie. <br>
A crew member is solely identified by his full name.

In [ ]:
crew_df.head()

In [ ]:
crew_df.shape

## Genres

In [ ]:
# Import 'genres.csv' dataset
genres_df = pd.read_csv('datasets/genres.csv')

### 1. Data Understanding

In [ ]:
genres_df.head()

In [ ]:
genres_df.shape

In [ ]:
genres_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
genres_df = genres_df.rename(columns={'id': 'movie_id'})
print(f"Genres dataset columns: {', '.join(genres_df.columns)}")

In [ ]:
# Check for null values
genres_df.isna().sum()

In [ ]:
# Check for duplicate rows
print(f"There are {genres_df.duplicated().sum()} duplicated rows")

In [ ]:
# Setting the correct type for columns
genres_list = list(genres_df["genre"].unique())
print(f'There are {len(genres_list)} genres in the dataset: {", ".join(genres_list)}')

genres_df['genre'] = genres_df['genre'].astype('category')

### 3. Final Results
The **genres** dataset is connected to the movies dataset through the 'movie_id' column. A movie can have multiple genres.


In [ ]:
genres_df.head()

In [ ]:
genres_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    genres_df.to_csv('clean_datasets/genres.csv')

genres_df = None

## Posters

In [ ]:
# Import 'posters.csv' dataset
posters_df = pd.read_csv('datasets/posters.csv')

### 1. Data Understanding

In [ ]:
posters_df.head()

In [ ]:
posters_df.shape

In [ ]:
posters_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
posters_df = posters_df.rename(columns={'id': 'movie_id', 'link': 'poster_link'})
print(f"Posters dataset columns: {', '.join(posters_df.columns)}")

In [ ]:
# Check for null values
posters_df.isna().sum()

In [ ]:
# Removing null rows
posters_df = posters_df.dropna()

It was decided to remove the `NaN` values as they do not contribute meaningful information to the dataset and could hinder data consistency and analysis.

In [ ]:
# Check for duplicate rows
print(f"There are {posters_df.duplicated().sum()} duplicated rows")

In [ ]:
# Check if a movie can have more than 1 poster
id_duplicates = posters_df['movie_id'].duplicated().any()

if id_duplicates:
    print("There are duplicates in the 'movie_id' column.")
else:
    print("There are no duplicates in the 'movie_id' column.")

The relationship between the **posters** dataset and the **movies** dataset must be One-to-One, allowing us to consider merging the two datasets.

In [ ]:
# Check for invalid links
link_regex = r'\bhttps?:\/\/(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}(?:\/[^\s]*)?'

all_valid = posters_df['poster_link'].str.contains(link_regex, na=False).all()

if all_valid:
    print("All links are valid.")
else:
    print("Some rows contains invalid links")

### 3. Final Results

In [ ]:
posters_df.head()

In [ ]:
posters_df.shape

In [ ]:
# TODO: Merging the datasets on 'id' from 'movies' and 'movie_id' from 'posters'
# merged_df = pd.merge(movies_df, posters_df, left_on='id', right_on='movie_id', how='left')

In [ ]:
if PRINT_CSV:
    posters_df.to_csv('clean_datasets/posters.csv')

posters_df = None

## Releases

In [ ]:
# Import 'release.csv' dataset
releases_df = pd.read_csv('datasets/releases.csv')

### 1. Data Understanding

In [ ]:
releases_df.head()

In [ ]:
releases_df.shape

In [ ]:
releases_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
releases_df = releases_df.rename(columns={'id': 'movie_id', 'type': 'distribution_format'})
print(f"Release dataset columns: {', '.join(releases_df.columns)}")

In [ ]:
# Check for null values
releases_df.isna().sum()

It's fine for the dataset to have null values only in the 'rating' field, we can keep the data without that information as the absence of ratings doesn't affect the overall analysis or integrity of the other data columns.

In [ ]:
# Check for duplicate rows
print(f"There are {releases_df.duplicated().sum()} duplicated rows")

In [ ]:
# Check for alternative null values in the dataset
releases_df[(releases_df['rating'] == "0") & (~releases_df['country'].isin(["Germany", "Austria", "Switzerland"]))].head()

We initially checked the value `0` in the dataset, assuming it might represent a null or missing value. However, we discovered that in Germany, Austria, and Switzerland, the `0` rating has a meaningful interpretation, indicating that the film is suitable for all audiences, including children. For the remaining countries, it is possible that the `0` rating is an error, but since it appears only 70 times in a dataset of 1,332,782 rows, we deemed it unnecessary to remove or correct these entries.

In [ ]:
# Setting the correct type for the columns
releases_df['date'] = pd.to_datetime(releases_df['date'], format='%Y-%m-%d')
releases_df['distribution_format'] = releases_df['distribution_format'].astype('category')

### 3. Final Results
The **release** dataset is linked to the movies dataset via the 'movie_id' column. It contains details about the movies' releases around the world.

In [ ]:
releases_df.head()

In [ ]:
releases_df.shape

In [ ]:
if PRINT_CSV:
    releases_df.to_csv('clean_datasets/releases.csv')

releases_df = None

## Studios

In [ ]:
# Import 'studios.csv' dataset
studios_df = pd.read_csv('datasets/studios.csv')

### 1. Data Understanding

In [ ]:
studios_df.head()

In [ ]:
studios_df.shape

In [ ]:
studios_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
studios_df = studios_df.rename(columns={'id': 'movie_id'})
print(f"Studios dataset columns: {', '.join(studios_df.columns)}")

In [ ]:
# Check for null values
studios_df.isna().sum()

In [ ]:
# Check for duplicate rows
print(f"There are {studios_df.duplicated().sum()} duplicated rows")
studios_df[studios_df.duplicated(keep=False)].head(6)

In [ ]:
studios_df = studios_df.dropna()
studios_df = studios_df.drop_duplicates()

We can safely remove the null and the duplicates values from the dataset, as they were not meaningful and irrelevant

### 3. Final Results
The **studios** dataset is connected to the movies dataset through the 'movie_id' column. It lists all the studios involved in each movie, allowing a movie to be associated with multiple studios and a studio to collaborate on multiple movies.

In [ ]:
studios_df.head()

In [ ]:
studios_df.shape

In [ ]:
if PRINT_CSV:
    studios_df.to_csv('clean_datasets/studios.csv')

studios_df = None

## Themes

In [ ]:
# Import 'themes.csv' dataset
themes_df = pd.read_csv('datasets/themes.csv')

### 1. Data Understanding

In [ ]:
themes_df.head()

In [ ]:
themes_df.shape

In [ ]:
themes_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
themes_df = themes_df.rename(columns={'id': 'movie_id'})
print(f"Themes dataset columns: {', '.join(themes_df.columns)}")

In [ ]:
# Check for null values
themes_df.isna().sum()

In [ ]:
# Check for duplicate rows
print(f"There are {themes_df.duplicated().sum()} duplicated rows")

In [ ]:
# Check for invalid themes
themes_df['theme'].value_counts().head(10)

### 3. Final Results
The **themes** dataset is linked to the movies dataset via the 'movie_id' column. It describes a movie with few, standard phrases.

In [ ]:
themes_df.head()

In [ ]:
themes_df.shape

In [ ]:
if PRINT_CSV:
    themes_df.to_csv('clean_datasets/themes.csv')
    
themes_df = None

## The Oscar Awards

In [ ]:
# Import 'the_oscar_awards.csv' dataset
oscars_df = pd.read_csv('datasets/the_oscar_awards.csv')

### 1. Data Understanding

In [ ]:
oscars_df.head()

In [ ]:
oscars_df.shape

In [ ]:
oscars_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
oscars_df = oscars_df.rename(columns={'name': 'nominee_name', 'film': 'nominated_film', 'winner': 'is_winner'})
print(f"Oscar dataset columns: {', '.join(oscars_df.columns)}")

In [ ]:
# Check for null values
oscars_df.isna().sum()

In [ ]:
# Check for duplicate rows
print(f"There are {oscars_df.duplicated().sum()} duplicated rows")
oscars_df[oscars_df.duplicated(keep=False)].head(6)

Duplicates are retained because, in music categories, the same artists can receive identical nominations for different songs, with the song titles not specified in the dataset

In [ ]:
# Check the consistency between year_film and year_ceremony
inconsistent_years_df = oscars_df[oscars_df['year_film'] > oscars_df['year_ceremony']]

if inconsistent_years_df.empty:
    print("The years fields are always consistent (ceremony year always greater than year film)")
else:
    inconsistent_years_df.head()

In [ ]:
# Check for multiple winner possibilities
from utils.utils import special_oscar_awards

# Exclude rows where the 'category' is in the special_oscar_awards list
filtered_oscars = oscars_df[~oscars_df['category'].isin(special_oscar_awards)]

# Find groups with more than one winner
multiple_winners = filtered_oscars.groupby(['year_ceremony', 'category']).filter(
    lambda x: x['is_winner'].sum() > 1
)

# Keep only the rows where 'is_winner' is True
multiple_winners = multiple_winners[multiple_winners['is_winner'] == True]
multiple_winners.head()

Displaying multiple winners (including draws), as we want to keep all winners in case of a draw in the same category (1969 Best Actress, or 1932 Best Actor). <br>
We decided to exclude some special awards, like the Jean Hersholt Humanitarian Award, because they are often less relevant to films and typically have multiple recipients. These awards are not traditional competitive awards but rather honorary recognitions.

In [ ]:
# Setting the correct type for columns
oscars_df['category'] = oscars_df['category'].astype('category')

### 3. Final Results

In [ ]:
oscars_df.head()

In [ ]:
oscars_df.shape

In [ ]:
if PRINT_CSV:
    oscars_df.to_csv('clean_datasets/the_oscar_awards.csv')

oscars_df = None

## Rotten Tomatoes Reviews

In [ ]:
# Import 'rotten_tomatoes_reviews' dataset
reviews_df = pd.read_csv('datasets/rotten_tomatoes_reviews.csv')

### 1. Data Understanding

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.shape

In [ ]:
reviews_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
reviews_df = reviews_df.rename(columns={'review_type': 'type', 'review_score': 'score', 'review_date': 'date', 'review_content': 'content', 'top_critic': 'is_top_critic'})
print(f"Rotten Tomatoes Reviews dataset columns: {', '.join(reviews_df.columns)}")

In [ ]:
# Check for null values
reviews_df.isna().sum()

Checking on Rotten Tomatoes website is fine having the publisher and critic name and content as null values.

In [ ]:
# Check for duplicate rows
filtered_df = reviews_df[reviews_df['critic_name'].notna()]
filtered_df = filtered_df[filtered_df.duplicated(keep=False)]

reviews_df = reviews_df.drop(reviews_df[reviews_df['critic_name'].isna()].index)

filtered_df

There are many duplicate reviews in the datasets. Looking closely is actually possible to have more reviews for the same movie that have the same publisher and with the author not specified. Those rows will be excluded from the total count of duplicate rows and will not be removed. All other duplicate rows will be removed.

In [ ]:
# Setting the correct type for columns
reviews_df['type'] = reviews_df['type'].astype('category')
reviews_df['date'] = pd.to_datetime(reviews_df['date'], format='%Y-%m-%d')

### 3. Final Result

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    reviews_df.to_csv('clean_datasets/reviews.csv')

reviews_df = None

### Saving the cleaned datasets

In [ ]:
# Print the clean datasets to new csv files TODO
if PRINT_CSV:
    movies_df.to_csv('clean_datasets/movies.csv')
    lang_df.to_csv('clean_datasets/languages.csv')
    actors_df.to_csv('clean_datasets/actors.csv')
    countries_df.to_csv('clean_datasets/countries.csv')
    crew_df.to_csv('clean_datasets/crew.csv')